In [1]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt

#from sklearn.multioutput import MultiOutputRegressor
from src.algorithms.multioutput import MultiOutputRegressor, RegressorChain

from sklearn.utils import shuffle
from src.algorithms.XGBoost import XGBoost

from sklearn.metrics import mean_absolute_error

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

import src.data as data
import src.utility as utility

from src.preprocessing.sensors import preprocess
preprocess()

from src.preprocessing.events import preprocess
preprocess()

from src.preprocessing.create_base_dataset import create_base_dataset
create_base_dataset(4,3)

import src.preprocessing.speeds as speeds

speeds.preprocess(data.speeds_original('test'))

from src.preprocessing.merge_features_to_dataset import merge
from src.features.avg_speed_street import AvgSpeedStreet
from src.features.avg_speed_sensor import AvgSpeedSensor
from src.features.avg_speed_sensor_hour import AvgSpeedSensorHour

features_array = [
        AvgSpeedStreet,
        AvgSpeedSensor,
        AvgSpeedSensorHour,
    ]


merge(features_array)

In [9]:
#dataset_train = base_df.merge(data.base_dataset(), how='left')
X, Y = data.dataset(onehot=False, drop_index_columns=True)

caching merged dataset train


In [10]:
X

,EMERGENCY_LANE,LANES,ROAD_TYPE,EVENT_DETAIL,EVENT_TYPE,SPEED_AVG_-10,SPEED_AVG_-9,SPEED_AVG_-8,SPEED_AVG_-7,SPEED_AVG_-6,SPEED_AVG_-5,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_SD_-10,SPEED_SD_-9,SPEED_SD_-8,SPEED_SD_-7,SPEED_SD_-6,SPEED_SD_-5,SPEED_SD_-4,SPEED_SD_-3,SPEED_SD_-2,SPEED_SD_-1,SPEED_MAX_-10,SPEED_MAX_-9,SPEED_MAX_-8,SPEED_MAX_-7,SPEED_MAX_-6,SPEED_MAX_-5,SPEED_MAX_-4,SPEED_MAX_-3,SPEED_MAX_-2,SPEED_MAX_-1,SPEED_MIN_-10,SPEED_MIN_-9,SPEED_MIN_-8,SPEED_MIN_-7,SPEED_MIN_-6,SPEED_MIN_-5,SPEED_MIN_-4,SPEED_MIN_-3,SPEED_MIN_-2,SPEED_MIN_-1,N_VEHICLES_-10,N_VEHICLES_-9,N_VEHICLES_-8,N_VEHICLES_-7,N_VEHICLES_-6,N_VEHICLES_-5,N_VEHICLES_-4,N_VEHICLES_-3,N_VEHICLES_-2,N_VEHICLES_-1,WEATHER_-10,WEATHER_-9,WEATHER_-8,WEATHER_-7,WEATHER_-6,WEATHER_-5,WEATHER_-4,WEATHER_-3,WEATHER_-2,WEATHER_-1,DISTANCE_-10,DISTANCE_-9,DISTANCE_-8,DISTANCE_-7,DISTANCE_-6,DISTANCE_-5,DISTANCE_-4,DISTANCE_-3,DISTANCE_-2,DISTANCE_-1,TEMPERATURE_-10,TEMPERATURE_-9,TEMPERATURE_-8,TEMPERATURE_-7,TEMPERATURE_-6,TEMPERATURE_-5,TEMPERATURE_-4,TEMPERATURE_-3,TEMPERATURE_-2,TEMPERATURE_-1,MIN_TEMPERATURE_-10,MIN_TEMPERATURE_-9,MIN_TEMPERATURE_-8,MIN_TEMPERATURE_-7,MIN_TEMPERATURE_-6,MIN_TEMPERATURE_-5,MIN_TEMPERATURE_-4,MIN_TEMPERATURE_-3,MIN_TEMPERATURE_-2,MIN_TEMPERATURE_-1,MAX_TEMPERATURE_-10,MAX_TEMPERATURE_-9,MAX_TEMPERATURE_-8,MAX_TEMPERATURE_-7,MAX_TEMPERATURE_-6,MAX_TEMPERATURE_-5,MAX_TEMPERATURE_-4,MAX_TEMPERATURE_-3,MAX_TEMPERATURE_-2,MAX_TEMPERATURE_-1,avg_speed_street,avg_speed_sd_street,avg_speed_min_street,avg_speed_max_street,avg_speed_sensor,avg_speed_sd_sensor,avg_speed_min_sensor,avg_speed_max_sensor,avg_speed_sensor_hour_y_0,avg_speed_sd_sensor_hour_y_0,avg_speed_min_sensor_hour_y_0,avg_speed_max_sensor_hour_y_0,avg_speed_sensor_hour_y_1,avg_speed_sd_sensor_hour_y_1,avg_speed_min_sensor_hour_y_1,avg_speed_max_sensor_hour_y_1,avg_speed_sensor_hour_y_2,avg_speed_sd_sensor_hour_y_2,avg_speed_min_sensor_hour_y_2,avg_speed_max_sensor_hour_y_2,avg_speed_sensor_hour_y_3,avg_speed_sd_sensor_hour_y_3,avg_speed_min_sensor_hour_y_3,avg_speed_max_sensor_hour_y_3
0,1,2,1,11,Veicolo_in_avaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199.6642,199.3150,204.7759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.4457,52.0643,43.0659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,263.734,292.858,294.476,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.866,59.866,59.866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,129.0,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quasi Sereno,Quasi Sereno,Quasi Sereno,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.0,38.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,28.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,21.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,29.0,29.0,219.025972,30.043761,141.760948,300.015243,216.049946,33.737127,105.728962,314.915000,206.013172,34.720362,69.698462,310.798242,205.731756,33.927031,74.931378,307.959333,205.427963,32.486134,74.446180,308.256270,204.220232,31.783890,77.112813,307.420000
1,1,2,1,2,Manutenzione_opere_in_verde,218.0592,204.6379,205.0416,203.6222,198.8546,201.6258,203.3925,202.6841,206.1482,206.6643,41.9865,34.5372,35.7394,38.0670,37.7942,34.3128,33.5805,34.7813,31.8745,25.9138,325.218,331.690,284.768,279.914,263.734,302.566,294.476,296.094,288.004,289.622,59.866,63.102,59.866,59.866,59.866,59.866,59.866,59.866,59.866,140.766,240.0,248.0,233.0,237.0,203.0,267.0,245.0,246.0,259.0,276.0,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,Quasi Sereno,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,18.0,18.0,18.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,12.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,219.025972,30.043761,141.760948,300.015243,216.049946,33.737127,105.728962,314.915000,206.013172,34.720362,69.698462,310.798242,205.731756,33.927031,74.931378,307.959333,205.427963,32.486134,74.446180,308.256270,204.220232,31.783890,77.112813,307.420000
2,1,2,1,4,Ostacolo_in_carreggiata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,216.3577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.3266,NaN,

In [11]:
#X, Y = shuffle(X, Y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

In [20]:
from src.algorithms.XGBoost import XGBoost

X = X.drop(['EVENT_TYPE', 'WEATHER_-10', 'WEATHER_-9','WEATHER_-8', 'WEATHER_-7', 'WEATHER_-6', 'WEATHER_-5', 'WEATHER_-4', 'WEATHER_-3', 'WEATHER_-2', 'WEATHER_-1'],axis=1)

xgboost = XGBoost(params_dict={
    'X': X,
    'objective' :'reg:linear',
    'colsample_bytree' : 0.3,
    'learning_rate' : 0.1,
    'max_depth' : 5,
    'alpha' : 10,
    'n_estimators' : 10
})

model= MultiOutputRegressor(xgboost)
model.fit(X, Y)

KeyError: "['EVENT_TYPE' 'WEATHER_-10' 'WEATHER_-9' 'WEATHER_-8' 'WEATHER_-7'\n 'WEATHER_-6' 'WEATHER_-5' 'WEATHER_-4' 'WEATHER_-3' 'WEATHER_-2'\n 'WEATHER_-1'] not found in axis"

In [21]:
xgboost = XGBoost(params_dict={
    'X': X,
    'objective' :'reg:linear',
    'colsample_bytree' : 0.3,
    'learning_rate' : 0.1,
    'max_depth' : 5,
    'alpha' : 10,
    'n_estimators' : 10
})

model= MultiOutputRegressor(xgboost)
model.fit(X, Y)

xgboost: fitting...
[0]	validation_0-rmse:160.652
[1]	validation_0-rmse:144.825
[2]	validation_0-rmse:130.552
[3]	validation_0-rmse:117.786
[4]	validation_0-rmse:106.061
[5]	validation_0-rmse:95.6452
[6]	validation_0-rmse:86.2577
[7]	validation_0-rmse:77.7933
[8]	validation_0-rmse:70.1939
[9]	validation_0-rmse:63.4004
xgboost: fitting...
[0]	validation_0-rmse:161.137
[1]	validation_0-rmse:145.278
[2]	validation_0-rmse:130.857
[3]	validation_0-rmse:117.968
[4]	validation_0-rmse:106.308
[5]	validation_0-rmse:95.9073
[6]	validation_0-rmse:86.6416
[7]	validation_0-rmse:78.1313
[8]	validation_0-rmse:70.4764
[9]	validation_0-rmse:63.6261
xgboost: fitting...
[0]	validation_0-rmse:161.051
[1]	validation_0-rmse:145.16
[2]	validation_0-rmse:130.754
[3]	validation_0-rmse:117.696
[4]	validation_0-rmse:106.014
[5]	validation_0-rmse:95.5972
[6]	validation_0-rmse:86.2543
[7]	validation_0-rmse:77.8119
[8]	validation_0-rmse:70.2427
[9]	validation_0-rmse:63.4613
xgboost: fitting...
[0]	validation_0-rmse

MultiOutputRegressor(estimator=<src.algorithms.XGBoost.XGBoost object at 0x1a23288390>,
           n_jobs=None)

In [17]:
def evaluate(model, X_test, y_test):
    mask_test = np.all(y_test.notnull(), axis=1)
    print('number of valid samples:', (mask_test*1).sum())

    y_pred = model.predict(X_test[mask_test])
    return mean_absolute_error(y_test[mask_test], y_pred)

In [18]:
model = RegressorChain(xgboost)
model.fit(X, Y)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x1a21bc2b70>>
Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/xgboost/core.py", line 366, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


xgboost: fitting...
[0]	validation_0-rmse:160.652
[1]	validation_0-rmse:144.825
[2]	validation_0-rmse:130.552
[3]	validation_0-rmse:117.786
[4]	validation_0-rmse:106.061
[5]	validation_0-rmse:95.6452
[6]	validation_0-rmse:86.2577
[7]	validation_0-rmse:77.7933
[8]	validation_0-rmse:70.1939
[9]	validation_0-rmse:63.4004
xgboost: fitting...


ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields $step_0$

In [22]:
evaluate(model,X_test, y_test)

number of valid samples: 5682


ValueError: could not convert string to float: 'Gestione_viabilita'